#Laboratorio
##Análisis de Datos de Ventas Mensuales

En este espacio de Colab, procederemos a desorrollar los codigos para la carga, limpieza, extraccion y depuracion segun corresponda en cada caso, para el analisis de los datos correspondientes a ventas mensuales en Estados Unidos del año 2019, de articulos tecnologicos.

Para hacer este analisis, aprovecharemos algunos codigos vistos en las clases con el profesor, codigos nuevos, mezclando implementaciones hechas en clase junto con otras deducidas por el grupo, otras investigadas, haciendo un conjunto que logre llegar al desarrollo de todo el analisis solicitado.

En primer lugar, importaremos las librerias necesarias para los codigos a desarrollar.

In [1]:
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [2]:


# Cargar el DataFrame original desde el archivo CSV

ruta_de_acceso = '/content/drive/MyDrive/Colab Notebooks/Big data - Machine Learning 2023/Clase 3'

archivos = os.listdir(ruta_de_acceso)
archivos

dataframes = {}

for archivo in archivos:
  if archivo.endswith('.csv'): # Asegurarse de que solo se consideren archivos CSV
    mes = archivo.split('_')[3].split('.')[0] # Obtener el nombre del archivo sin la extensión
    ruta_completa = os.path.join(ruta_de_acceso, archivo) # Ruta completa al archivo
    df = pd.read_csv(ruta_completa)
    df_ventas = df.copy()

    df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors='coerce')  #El errors= 'coerce' transforma los valores no numericos a NaN'
    df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors='coerce')


    df_ventas = df_ventas.loc[~df_ventas['Cantidad Pedida'].isna()]
    df_ventas = df_ventas.loc[~df_ventas['Precio Unitario'].isna()]

     # Elimino todos los Nan y me quedo solo con los valores numericos
    df_ventas = df_ventas.dropna()

     # Transformo todos los valores numericos al tipo de dato int
    df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int)
    df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(int)


    df_ventas['Fecha de Compra'] = pd.to_datetime(df_ventas['Fecha de Pedido'])



    dataframes[mes] = df_ventas  # Agregar el DataFrame al diccionario usando el nombre del mes



In [3]:
df_ventas #hacemos un 'print' del dataframe para examinar como quedo luego de la depuracion.

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Fecha de Compra
0,194095,Wired Headphones,1,11,05/16/19 17:14,"669 2nd St, New York City, NY 10001",2019-05-16 17:14:00
1,194096,AA Batteries (4-pack),1,3,05/19/19 14:43,"844 Walnut St, Dallas, TX 75001",2019-05-19 14:43:00
2,194097,27in FHD Monitor,1,149,05/24/19 11:36,"164 Madison St, New York City, NY 10001",2019-05-24 11:36:00
3,194098,Wired Headphones,1,11,05/02/19 20:40,"622 Meadow St, Dallas, TX 75001",2019-05-02 20:40:00
4,194099,AAA Batteries (4-pack),2,2,05/11/19 22:55,"17 Church St, Seattle, WA 98101",2019-05-11 22:55:00
...,...,...,...,...,...,...,...
16630,209916,Bose SoundSport Headphones,1,99,05/08/19 19:15,"235 Center St, Seattle, WA 98101",2019-05-08 19:15:00
16631,209917,Wired Headphones,1,11,05/24/19 22:02,"926 9th St, San Francisco, CA 94016",2019-05-24 22:02:00
16632,209918,AAA Batteries (4-pack),2,2,05/24/19 17:44,"868 Hickory St, Boston, MA 02215",2019-05-24 17:44:00
16633,209919,USB-C Charging Cable,1,11,05/04/19 12:46,"204 Adams St, Seattle, WA 98101",2019-05-04 12:46:00




---



#1. Comportamiento de las ventas en los distintos meses:
En esta seccion generaremos los graficos correspondientes para responder las preguntas referentes a las ventas en los distintos meses del año.

In [4]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
        'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

for mes in meses:
    dataframe_mes = dataframes[mes]

#Hacemos una lista con los meses del año e iteramos con el diccionario de dataframes para que cada mes de la lista se asigne a los datos del dataframe de su correspondiente mes.

In [5]:
def extraer_datos(datos):
  datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
  datos['Hora'] = datos['Fecha de Pedido'].dt.hour
  datos['Dia'] = datos['Fecha de Pedido'].dt.day
  datos['Mes'] = datos['Fecha de Pedido'].dt.month
  return datos

#Funcion para exraer datos de los dataframes vista en clases.

En primer lugar, implementamos el grafico hecho en clases para ver las ventas de cada mes.

In [6]:

# Creo listas para almacenar los totales de ventas y los nombres de los meses
totales_ventas = []
nombres_meses = []

for mes in meses:
    dataframe_mes = dataframes[mes]
    dataframe_mes = extraer_datos(dataframe_mes)

    total_ventas_mes = dataframe_mes['Cantidad Pedida'].sum()
    totales_ventas.append(total_ventas_mes)
    nombres_meses.append(mes)

# Creo el objeto figura con el gráfico de barras
fig = go.Figure()

fig.add_trace(go.Bar(
    x=nombres_meses,
    y=totales_ventas,
    marker=dict(color=totales_ventas, colorscale = 'RdYlGn', colorbar=dict(title="Total de ventas", x=1.0, thickness=15)),  # Color y barra de colores
    text=totales_ventas,
    textposition='auto',  # Muestro el valor encima de las barras

))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Total de ventas por mes",
    xaxis=dict(title="Mes"),
    yaxis=dict(title="Total de ventas"),
)

fig.show()

Podemos ver en este grafico, las ventas de cada mes y las primeras observaciones que se pueden hacer es que diciembre fue el mes con mas ventas con una diferencia bastante notoria respecto a los demas.

Ahora, desarrollamos un grafico con una dinamica similar, pero en este caso de linea, para poder tener otra perspectiva de la variacion de ventas a lo largo de los meses.

In [7]:
import pandas as pd
import plotly.graph_objects as go



# Creo listas para almacenar los totales de ventas y los nombres de los meses
totales_ventas = []
nombres_meses = []

for mes in meses:
    dataframe_mes = dataframes[mes]
    dataframe_mes = extraer_datos(dataframe_mes)

    total_ventas_mes = dataframe_mes['Cantidad Pedida'].sum()
    totales_ventas.append(total_ventas_mes)
    nombres_meses.append(mes)

# Creo el objeto figura con el gráfico de línea
fig = go.Figure()

fig.add_trace(go.Line(
    x=nombres_meses,
    y=totales_ventas,
    marker=dict(color=totales_ventas, colorscale='RdYlGn', colorbar=dict(title="Total de ventas")),  # Color y barra de colores
    text=totales_ventas,
    mode='lines+markers'  # Mostrar puntos de datos en la línea
))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Variación de Ventas Mensuales",
    xaxis=dict(title="Mes"),
    yaxis=dict(title="Total de ventas"),
)

fig.show()








/usr/local/lib/python3.10/dist-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




En este grafico de linea, notamos la variacion a lo largo de los meses, siendo Enero el mes con menores ventas y Diciembre con la mayor cantidad de ventas, pero podemos observar que a lo largo hay un crecimiento desde Enero hasta Abril, que luego disminuye hasta junio, manteniendose un poco mas estable pero con tendencia a la baja hasta Septiembre, ahi es donde se ve una subida muy notoria hasta Octubre, bajando un poco hacia noviembre, y alcanzando su pico maximo desde dicho mes a Diciembre, que, como dicho anteriormente, es el mes destacado con mayores ventas del año.



---



Para poder visualizar cual es el ingreso total generado por mes, generamos un grafico similar al de ventas mensuales, pero esta vez con los ingresos brutos de dichos meses

In [8]:
import pandas as pd
import plotly.graph_objects as go

# Creo listas para almacenar los totales de ingresos y los nombres de los meses
totales_ingresos = []
nombres_meses = []

for mes in meses:
    dataframe_mes = dataframes[mes]
    dataframe_mes = extraer_datos(dataframe_mes)

    # Calcula los ingresos multiplicando la cantidad pedida por el precio unitario
    ingresos_mes = dataframe_mes['Cantidad Pedida'] * dataframe_mes['Precio Unitario']
    total_ingresos_mes = ingresos_mes.sum()
    totales_ingresos.append(total_ingresos_mes)
    nombres_meses.append(mes)

# Creo el objeto figura con el gráfico de barras
fig = go.Figure()

fig.add_trace(go.Bar(
    x=nombres_meses,
    y=totales_ingresos,
    marker=dict(color=totales_ingresos, colorscale='YlGnBu', colorbar=dict(title="Total de ingresos", x=1.0, thickness=15)),  # Color y barra de colores
    text=totales_ingresos,
    textposition='auto',  # Muestro el valor encima de las barras
))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Total de ingresos por mes",
    xaxis=dict(title="Mes"),
    yaxis=dict(title="Total de ingresos"),
)

fig.show()

En este grafico podemos ver los ingresos de cada mes, siendo coincidentes con la cantidad de ventas vistas en los graficos anteriores.



---



Ademas, hicimos un grafico de torta para poder apreciar mejor el porcentaje de ingresos segun cada mes.

In [9]:
# Calcula los porcentajes de ingresos para cada mes
porcentajes_ingresos = [(ingreso / sum(totales_ingresos)) * 100 for ingreso in totales_ingresos]

# Crear un DataFrame con los porcentajes
df_porcentajes = pd.DataFrame({'Mes': nombres_meses, 'Porcentaje Ingresos': porcentajes_ingresos})

# Crear el gráfico de torta
fig = go.Figure(data=[go.Pie(labels=df_porcentajes['Mes'], values=df_porcentajes['Porcentaje Ingresos'])])

# Configurar el título
fig.update_layout(
    title="Porcentaje de Ingresos por Mes",
)

fig.show()




---



Luego de analizar todos estos graficos, se podria decir que los tres meses mas destacados son:
Abril, ya que si bien no es mucha la diferencia, registra mas
ventas que todos sus meses antecesores y hasta su mes posterior.
Octubre: Saca una diferencia altisima respecto al mes anterior, Septiembre, y es el segundo mes con mas ventas del año.
Diciembre: Como ya se ha mencionado anteriormente, Diciembre es el mes con mas ventas anuales.



---



---



#2. Optimización de la publicidad y patrón de ventas por hora:

Para trabajar con los datos de todos los meses juntos, procederemos a realizar una concatenacion de los 12 dataframes con sus meses correspondientes, y luego hacer un desglose como vimos en clase, de algunas columnas del dataframe, para poder trabajarlos mas eficientemente.

In [10]:
#Concatenamos los dataframes de todos los meses.
df_anual = pd.concat([
    dataframes['Enero'],
    dataframes['Febrero'],
    dataframes['Marzo'],
    dataframes['Abril'],
    dataframes['Mayo'],
    dataframes['Junio'],
    dataframes['Julio'],
    dataframes['Agosto'],
    dataframes['Septiembre'],
    dataframes['Octubre'],
    dataframes['Noviembre'],
    dataframes['Diciembre']])


In [11]:
#Desglosamos algunas columnas del dataframe
# Separo la direccion de envio en calle ciudad y PO BOX
direccion_df = df_anual['Dirección de Envio'].str.split(', ',expand=True)
direccion_df.columns = ['Calle','Ciudad','PO BOX']

#Separo el PO BOX en Estado y Cod Postal
direccion_df[['Estado', 'Cod Postal']] = direccion_df['PO BOX'].str.split(' ',expand=True)
df_ventas_anual_desglosado = pd.concat([df_anual,direccion_df],axis=True)

#Concatenamos los dataframes de todos los meses.
df_anual = pd.concat([
    dataframes['Enero'],
    dataframes['Febrero'],
    dataframes['Marzo'],
    dataframes['Abril'],
    dataframes['Mayo'],
    dataframes['Junio'],
    dataframes['Julio'],
    dataframes['Agosto'],
    dataframes['Septiembre'],
    dataframes['Octubre'],
    dataframes['Noviembre'],
    dataframes['Diciembre']])

#Desglosamos algunas columnas del dataframe
# Separo la direccion de envio en calle ciudad y PO BOX
direccion_df = df_anual['Dirección de Envio'].str.split(', ',expand=True)
direccion_df.columns = ['Calle','Ciudad','PO BOX']

#Separo el PO BOX en Estado y Cod Postal
direccion_df[['Estado', 'Cod Postal']] = direccion_df['PO BOX'].str.split(' ',expand=True)
df_ventas_anual_desglosado = pd.concat([df_anual,direccion_df],axis=True)



In [12]:
df_ventas_anual_desglosado

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Fecha de Compra,Hora,Dia,Mes,Calle,Ciudad,PO BOX,Estado,Cod Postal
0,141234,iPhone,1,700,2019-01-22 21:25:00,"944 Walnut St, Boston, MA 02215",2019-01-22 21:25:00,21,22,1,944 Walnut St,Boston,MA 02215,MA,02215
1,141235,Lightning Charging Cable,1,14,2019-01-28 14:15:00,"185 Maple St, Portland, OR 97035",2019-01-28 14:15:00,14,28,1,185 Maple St,Portland,OR 97035,OR,97035
2,141236,Wired Headphones,2,11,2019-01-17 13:33:00,"538 Adams St, San Francisco, CA 94016",2019-01-17 13:33:00,13,17,1,538 Adams St,San Francisco,CA 94016,CA,94016
3,141237,27in FHD Monitor,1,149,2019-01-05 20:33:00,"738 10th St, Los Angeles, CA 90001",2019-01-05 20:33:00,20,5,1,738 10th St,Los Angeles,CA 90001,CA,90001
4,141238,Wired Headphones,1,11,2019-01-25 11:59:00,"387 10th St, Austin, TX 73301",2019-01-25 11:59:00,11,25,1,387 10th St,Austin,TX 73301,TX,73301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25112,319666,Lightning Charging Cable,1,14,2019-12-11 20:58:00,"14 Madison St, San Francisco, CA 94016",2019-12-11 20:58:00,20,11,12,14 Madison St,San Francisco,CA 94016,CA,94016
25113,319667,AA Batteries (4-pack),2,3,2019-12-01 12:01:00,"549 Willow St, Los Angeles, CA 90001",2019-12-01 12:01:00,12,1,12,549 Willow St,Los Angeles,CA 90001,CA,90001
25114,319668,Vareebadd Phone,1,400,2019-12-09 06:43:00,"273 Wilson St, Seattle, WA 98101",2019-12-09 06:43:00,6,9,12,273 Wilson St,Seattle,WA 98101,WA,98101
25115,319669,Wired Headphones,1,11,2019-12-03 10:39:00,"778 River St, Dallas, TX 75001",2019-12-03 10:39:00,10,3,12,778 River St,Dallas,TX 75001,TX,75001


Ahora tenemos un dataframe concatenado con los datos de todos los meses , y desglosado en varias columnas mas como 'Ciudad', 'Estado', y otras como 'Dia' y 'Mes' que se han generado con la funcion extraer_datos.



---



Ahora generaremos los graficos para analizar los pedidos globales segun la hora del dia.

In [13]:
# Contamos la cantidad de pedidos por hora
conteo_pedidos_por_hora = df_ventas_anual_desglosado['Hora'].value_counts().sort_index()

# Creamos el gráfico de barras con Plotly
fig = px.bar(x=conteo_pedidos_por_hora.index, y=conteo_pedidos_por_hora.values, labels={'x': 'Hora del día', 'y': 'Cantidad de pedidos'}, title='Cantidad de pedidos por hora del día')

# Personalizamos el eje x con etiquetas específicas para las horas
fig.update_layout(xaxis=dict(tickvals=conteo_pedidos_por_hora.index, ticktext=conteo_pedidos_por_hora.index))

fig.show()

Con este grafico, podemos observar los pedidos que se hacen a lo largo de todas las horas del dia, pudiendo notarse que el primer pico de ventas se da entre las 11 y las 13 horas, y el segundo pico se da entre las 18 y las 20 horas, siendo las 19 la hora pico del dia en cuanto a compras.



---



Creamos un grafico tipo mapa de calor, para poder observar mejor este patron a lo largo de los meses.

In [14]:
import pandas as pd
import plotly.express as px




#  Agrupamos los datos por 'Mes' y 'Hora' y suma las cantidades pedidas en cada grupo
ventas_por_hora = df_ventas_anual_desglosado.groupby(['Mes', 'Hora'])['Cantidad Pedida'].sum().reset_index()

#  Creamos un gráfico de calor (heatmap)
fig = px.imshow(ventas_por_hora.pivot('Mes', 'Hora', 'Cantidad Pedida'),
                labels=dict(x="Hora del Día", y="Mes del Año", color="Ventas"),
                x=ventas_por_hora['Hora'].unique(),
                y=ventas_por_hora['Mes'].unique(),
                color_continuous_scale="Viridis",  # Puedes cambiar la paleta de colores
                title="Patrones de Ventas por Hora a lo largo del Año")

# Configuramos el eje X
fig.update_xaxes(title_text="Hora del Día", side="top")
fig.update_xaxes(nticks=24, tickvals=list(range(24)), ticktext=[str(i) for i in range(24)])
fig.update_yaxes(title_text="Mes del Año", tickvals=list(range(1, 13)), ticktext=["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"])

fig.show()

<ipython-input-14-4abb6b337812>:11: FutureWarning:

In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.



Con este mapa de calor, vemos que los cuadrados mas calidos/amarillentos/verdosos nos indican las horas con mayores ventas, y los cuadrados mas obscuros/azulados las horas con menores ventas, y si observamos los cuadrados mas calidos se encuentran a lo largo de todos los meses en aproximadamente las mismas horas vistas en el grafico anterior de barras, es decir, entre las 10 y las 13 de la tarde, y un segundo pico entre las 17 y las 20 horas, aproximadamente y promediando todos los meses.



---



---



#3. Distribución de ventas por ubicación:

##Ventas por ciudad

*   Generamos un sub dataframe y agrupamos las ciudades con la suma de sus respectivas ventas.
*   Hacemos el grafico correspondiente al dataframe generado con estos datos de ventas por ciudad.



In [15]:
for mes in meses:
  ventas_por_ciudad = df_ventas_anual_desglosado.groupby('Ciudad')['Cantidad Pedida'].sum().reset_index()
ventas_por_ciudad

,Ciudad,Cantidad Pedida
0,Atlanta,16602
1,Austin,11153
2,Boston,22528
3,Dallas,16730
4,Los Angeles,33289
5,New York City,27932
6,Portland,14053
7,San Francisco,50239
8,Seattle,16553


In [16]:
import plotly.express as px
fig = px.bar(ventas_por_ciudad, x='Ciudad', y='Cantidad Pedida', text='Cantidad Pedida',
             labels={'Cantidad Pedida': 'Ventas'},
             title='Ventas por Ciudad')



fig.update_traces(marker_color=['blue', 'green', 'red', 'purple', 'orange', 'pink', 'brown', 'gray', 'cyan'],texttemplate='%{text}', textposition='outside',marker_line_width=2, marker_line_color='black', opacity=0.8) # Muestra el valor de ventas en las barras
fig.update_xaxes(title='Ciudad')
fig.update_xaxes(categoryorder='total descending')
fig.update_yaxes(title='Ventas')

fig.update_layout(
    title='Ventas por Ciudad',
    xaxis_title='Ciudad',
    yaxis_title='Ventas',
    plot_bgcolor='lightgray')

fig.show()



---



##Ventas por Estado

*   Generamos un sub dataframe y agrupamos los estados con la suma de sus respectivas ventas.
*   Hacemos el grafico correspondiente al dataframe generado con estos datos de ventas por estado.



In [17]:
for mes in meses:
  ventas_por_estado = df_ventas_anual_desglosado.groupby('Estado')['Cantidad Pedida'].sum().reset_index()
ventas_por_estado

,Estado,Cantidad Pedida
0,CA,83528
1,GA,16602
2,MA,22528
3,ME,2750
4,NY,27932
5,OR,11303
6,TX,27883
7,WA,16553


In [18]:
fig = px.choropleth(ventas_por_estado, locations='Estado', locationmode='USA-states',color='Cantidad Pedida',
                           color_continuous_scale="Viridis",
                           scope="usa",
                          title= 'Ventas por estado'
                          )

fig.show()



---



Realizamos un grafico de las ventas por estado a lo largo de los meses.

In [19]:
ventas_por_estado_mes = df_ventas_anual_desglosado.groupby(['Estado', 'Mes'])['Cantidad Pedida'].sum().reset_index()

In [20]:
nombres_meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

In [21]:
ventas_por_estado_mes['NombreMes'] = [nombres_meses[num - 1] for num in ventas_por_estado_mes['Mes']] #En esta linea de codigo creamos una columna nueva con el nombre del mes correspondiente a cada numero de mes.

# Crea el gráfico utilizando los nombres de meses en el eje X
fig = px.line(ventas_por_estado_mes, x='NombreMes', y='Cantidad Pedida', color='Estado',
              title='Ventas por Estado a lo largo de los Meses',
              labels={'Ventas': 'Ventas', 'NombreMes': 'Mes'})
fig.show()



---



---



#4. Análisis del producto más vendido.

Generamos un dataframe con los productos mas vendidos globalmente.

Con este dataframe, hacemos el grafico correspondiente que nos muestra las ventas de todos los productos.

In [22]:
Productos_vendidos = df_ventas_anual_desglosado.groupby('Producto')['Cantidad Pedida'].sum()


# Ordenar de mayor a menor
Productos_vendidos_ordenados = Productos_vendidos.sort_values(ascending = False)

#Crear un nuevo DataFrame con los valores
df_Productos = pd.DataFrame({'Producto': Productos_vendidos_ordenados.index, 'Cantidad Pedida': Productos_vendidos_ordenados.values})

df_Productos

,Producto,Cantidad Pedida
0,AAA Batteries (4-pack),31017
1,AA Batteries (4-pack),27635
2,USB-C Charging Cable,23975
3,Lightning Charging Cable,23217
4,Wired Headphones,20557
5,Apple Airpods Headphones,15661
6,Bose SoundSport Headphones,13457
7,27in FHD Monitor,7550
8,iPhone,6849
9,27in 4K Gaming Monitor,6244


In [23]:
Productos = df_Productos['Producto']
Unidades =  df_Productos['Cantidad Pedida']

#Creo el objeto figura
fig = go.Figure()

#agrego grafico en mi objeto figura
fig.add_trace(go.Bar(
              x = Productos,
              y= Unidades,
              name = 'Unidades Vendidas',
              marker = dict(color= Unidades, colorscale = 'RdYlGn', colorbar = dict(title= 'Total de ventas') )
)
)

fig.update_layout(
    title= 'Unidades Vendidas',
    xaxis = dict(title= 'Productos'),
    yaxis = dict(title= 'Unidades'))

Este grafico nos muestra de mayor a menor el producto mas vendido, donde se puede apreciar que las baterias AAA son globalmente el articulo mas solicitado.

Ahora, para saber cual es el producto mas vendido por mes, realizamos un  codigo que nos genere un dataframe que muestre el top 3 de los productos mas vendidos de cada mes, y luego con ese dataframe, graficamos en barras apiladas.

In [24]:
ventas_por_mes_producto = df_ventas_anual_desglosado.groupby(['Mes', 'Producto'])['Cantidad Pedida'].sum().reset_index()

In [25]:
ventas_por_mes_producto

,Mes,Producto,Cantidad Pedida
0,1,20in Monitor,218
1,1,27in 4K Gaming Monitor,313
2,1,27in FHD Monitor,422
3,1,34in Ultrawide Monitor,314
4,1,AA Batteries (4-pack),1424
...,...,...,...
223,12,ThinkPad Laptop,539
224,12,USB-C Charging Cable,3250
225,12,Vareebadd Phone,284
226,12,Wired Headphones,2749


In [26]:
ventas_por_mes_producto['NombreMes'] = [nombres_meses[num - 1] for num in ventas_por_mes_producto['Mes']]

In [27]:
ventas_por_mes_producto

,Mes,Producto,Cantidad Pedida,NombreMes
0,1,20in Monitor,218,Enero
1,1,27in 4K Gaming Monitor,313,Enero
2,1,27in FHD Monitor,422,Enero
3,1,34in Ultrawide Monitor,314,Enero
4,1,AA Batteries (4-pack),1424,Enero
...,...,...,...,...
223,12,ThinkPad Laptop,539,Diciembre
224,12,USB-C Charging Cable,3250,Diciembre
225,12,Vareebadd Phone,284,Diciembre
226,12,Wired Headphones,2749,Diciembre


In [28]:
top_3_productos_por_mes = ventas_por_mes_producto.groupby(['NombreMes', 'Producto'])['Cantidad Pedida'].sum().reset_index()
top_3_productos_por_mes = top_3_productos_por_mes.sort_values(['NombreMes', 'Cantidad Pedida'], ascending=[True, False])
top_3_productos_por_mes = top_3_productos_por_mes.groupby('NombreMes').head(3)

top_3_productos_por_mes

,NombreMes,Producto,Cantidad Pedida
5,Abril,AAA Batteries (4-pack),2939
4,Abril,AA Batteries (4-pack),2821
12,Abril,Lightning Charging Cable,2373
24,Agosto,AAA Batteries (4-pack),2021
23,Agosto,AA Batteries (4-pack),1828
34,Agosto,USB-C Charging Cable,1462
43,Diciembre,AAA Batteries (4-pack),4241
42,Diciembre,AA Batteries (4-pack),3724
53,Diciembre,USB-C Charging Cable,3250
62,Enero,AAA Batteries (4-pack),1600


In [29]:
fig = px.bar(top_3_productos_por_mes, x='NombreMes', y='Cantidad Pedida', color='Producto',
             title='Top 3 Productos Más Vendidos por Mes',
             labels={'Cantidad Pedida': 'Cantidad Vendida', 'NombreMes': 'Mes'},
             category_orders={"NombreMes": ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
                                             'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']},
             text='Cantidad Pedida')
fig.update_traces(texttemplate='%{text}', textposition='inside')  # Colocar el texto dentro de la barra

fig.show()

Al igual que el grafico que nos mostraba de forma global, en este que estan los productos mas vendidos por cada mes, tambien, y de forma unanime en todos los meses, el producto que se encuentra en el primer lugar del top mas vendidos son las baterias AAA



---



Ahora, algo a tener en cuenta a la hora de analizar el exito de un producto en particular y porque tiene tantas ventas, puede ser el precio, de manera que procedemos a hacer un grafico ya visto en clases que relacione el producto con el precio, pero adaptado a nuestro dataframe global.

In [30]:
#Extraigo valores del dataframe para realizar el grafico
Analisis_de_productos =  df_ventas_anual_desglosado.groupby('Producto').agg({
    'Cantidad Pedida': 'sum',
    'Precio Unitario': 'mean'
})

#Ordeno de mayor a menor
Analisis_de_productos = Analisis_de_productos.sort_values(by='Cantidad Pedida', ascending= False)

#Reseteo indice
Analisis_de_productos = Analisis_de_productos.reset_index(inplace=False)


Analisis_de_productos



,Producto,Cantidad Pedida,Precio Unitario
0,AAA Batteries (4-pack),31017,2.0
1,AA Batteries (4-pack),27635,3.0
2,USB-C Charging Cable,23975,11.0
3,Lightning Charging Cable,23217,14.0
4,Wired Headphones,20557,11.0
5,Apple Airpods Headphones,15661,150.0
6,Bose SoundSport Headphones,13457,99.0
7,27in FHD Monitor,7550,149.0
8,iPhone,6849,700.0
9,27in 4K Gaming Monitor,6244,389.0


In [31]:
Productos = Analisis_de_productos['Producto']
Unidades = Analisis_de_productos['Cantidad Pedida']
Precio = Analisis_de_productos ['Precio Unitario']

#Crear el objeto figura
fig = go.Figure()

fig.add_trace(go.Bar(
              x = Productos,
              y= Unidades,
              name = 'Unidades Vendidas',
              marker = dict(color= 'Light Blue'),
              yaxis='y'
)
)
fig.add_trace(go.Scatter(
              x = Productos,
              y= Precio,
              name = 'Precio unitario',
              line = dict(color= 'Red', shape='spline', smoothing = 1.3, width=3),
              yaxis='y2'
)
)

fig.update_layout(
    title= 'Unidades Vendidas y Precios de productos',
    xaxis = dict(title= 'Productos'),
    yaxis = dict(title= 'Unidades', side = 'left'),
    yaxis2 = dict(title='Precio', overlaying= 'y', side='right'),
    yaxis_range = [0, max(Unidades)*1.1],
    yaxis2_range = [0, max(Precio)*1.1]
)

Este grafico nos muestra claramente que hay una tendencia de ventas mayores mientras menor sea el precio, por lo menos con los productos en top 3 de ventas, porque por ejemplo si analizaramos mas en detalle, podemos ver que el iPhone es mas costoso que el Google Phone o la TV Flatscreen, pero sin embargo sus ventas son mayores a estos productos.



---



---



#5. Tendencia ventas

Para analizar las tendencias en cuanto a ventas a lo largo de los dias en todos los meses, primero propusimos hacer un grafico por cada mes, para poder observar mas detalladamente cuales eran los dias con mayores y menores ventas de cada uno.


Primero utilizamos la funcion extraer_datos para obtener de cada dataframe del diccionario de dataframes, los datos que vamos a necesitar para graficar lo que necesitamos, como el mes y el dia por ejemplo.

In [32]:
for mes in meses:
  dataframes[mes] = extraer_datos(dataframes[mes])


In [33]:
dataframes['Febrero'] #Ejemplo para poder analizar el dataframe con los datos ya extraidos, en este caso, seleccionamos el mes de Febrero.

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Fecha de Compra,Hora,Dia,Mes
0,150502,iPhone,1,700,2019-02-18 01:35:00,"866 Spruce St, Portland, ME 04101",2019-02-18 01:35:00,1,18,2
1,150503,AA Batteries (4-pack),1,3,2019-02-13 07:24:00,"18 13th St, San Francisco, CA 94016",2019-02-13 07:24:00,7,13,2
2,150504,27in 4K Gaming Monitor,1,389,2019-02-18 09:46:00,"52 6th St, New York City, NY 10001",2019-02-18 09:46:00,9,18,2
3,150505,Lightning Charging Cable,1,14,2019-02-02 16:47:00,"129 Cherry St, Atlanta, GA 30301",2019-02-02 16:47:00,16,2,2
4,150506,AA Batteries (4-pack),2,3,2019-02-28 20:32:00,"548 Lincoln St, Seattle, WA 98101",2019-02-28 20:32:00,20,28,2
...,...,...,...,...,...,...,...,...,...,...
12031,162004,Apple Airpods Headphones,1,150,2019-02-12 22:02:00,"227 Church St, San Francisco, CA 94016",2019-02-12 22:02:00,22,12,2
12032,162005,AAA Batteries (4-pack),2,2,2019-02-04 20:44:00,"417 Jefferson St, Los Angeles, CA 90001",2019-02-04 20:44:00,20,4,2
12033,162006,USB-C Charging Cable,1,11,2019-02-24 06:31:00,"498 8th St, Atlanta, GA 30301",2019-02-24 06:31:00,6,24,2
12034,162007,USB-C Charging Cable,1,11,2019-02-24 19:09:00,"715 7th St, Dallas, TX 75001",2019-02-24 19:09:00,19,24,2




---



#*Esta es quizas la parte mas repetitiva, donde haremos exactamente el mismo codigo para cada mes, para obtener un dataframe solamente con los dias y las ventas de cada respectivo mes.*

In [34]:
dataframe_enero = dataframes["Enero"].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_enero = dataframe_enero.reset_index(inplace=False)
dataframe_enero = dataframe_enero.sort_values(by='Cantidad Pedida', ascending=False)
dataframe_enero

,Dia,Cantidad Pedida
25,26,423
23,24,391
19,20,385
27,28,368
9,10,368
12,13,368
1,2,368
18,19,364
10,11,363
21,22,359




---



In [35]:
fig = px.bar(dataframe_enero, x='Dia', y='Cantidad Pedida',
             title='Ventas por Día en Enero',
             labels={'Cantidad Pedida': 'Cantidad Vendida', 'Dia': 'Día'},
             color='Cantidad Pedida',  # Usar el valor de ventas para la paleta de colores
             color_continuous_scale='Viridis')  # Elegir una paleta de colores

# Ajustar el rango del eje y para resaltar mejor las diferencias
fig.update_yaxes(range=[0, dataframe_enero['Cantidad Pedida'].max() * 1.2])
fig.update_layout( xaxis=dict( tickvals= dataframe_enero['Dia']))
fig.show()

Estos graficos estan diseñados para que veamos de colores mas amarillos a los dias con mas ventas, y mas obscuros/violetas a los dias de menores ventas, en este caso, el dia de mayores ventas de Enero, fue el 26.

Ahora repetimos este proceso con los demas meses...

In [36]:
dataframe_febrero = dataframes["Febrero"].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_febrero = dataframe_febrero.reset_index(inplace=False)
dataframe_febrero = dataframe_febrero.sort_values(by='Cantidad Pedida', ascending=False)
dataframe_febrero

,Dia,Cantidad Pedida
12,13,543
23,24,531
16,17,510
26,27,509
17,18,508
21,22,507
9,10,506
19,20,500
1,2,499
11,12,498


In [37]:
fig = px.bar(dataframe_febrero, x='Dia', y='Cantidad Pedida',
             title='Ventas por Día en Febrero',
             labels={'Cantidad Pedida': 'Cantidad Vendida', 'Dia': 'Día'},
             color='Cantidad Pedida',  # Usar el valor de ventas para la paleta de colores
             color_continuous_scale='Viridis')  # Elegir una paleta de colores

# Ajustar el rango del eje y para resaltar mejor las diferencias
fig.update_yaxes(range=[0, dataframe_febrero['Cantidad Pedida'].max() * 1.2])
fig.update_layout( xaxis=dict( tickvals= dataframe_febrero['Dia']))
fig.show()

El dia de mayores ventas de Febrero fue el 13.

In [38]:
dataframe_marzo = dataframes["Marzo"].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_marzo = dataframe_marzo.reset_index(inplace=False)
dataframe_marzo = dataframe_marzo.sort_values(by='Cantidad Pedida', ascending=False)
dataframe_marzo

,Dia,Cantidad Pedida
22,23,616
4,5,577
1,2,575
10,11,575
12,13,574
21,22,567
26,27,564
6,7,559
11,12,559
24,25,556


In [39]:
fig = px.bar(dataframe_marzo, x='Dia', y='Cantidad Pedida',
             title='Ventas por Día en Marzo',
             labels={'Cantidad Pedida': 'Cantidad Vendida', 'Dia': 'Día'},
             color='Cantidad Pedida',  # Usar el valor de ventas para la paleta de colores
             color_continuous_scale='Viridis')  # Elegir una paleta de colores

# Ajustar el rango del eje y para resaltar mejor las diferencias
fig.update_yaxes(range=[0, dataframe_marzo['Cantidad Pedida'].max() * 1.2])
fig.update_layout( xaxis=dict( tickvals= dataframe_marzo['Dia']))
fig.show()

El dia de mayores ventas de Marzo fue el 23.

In [40]:
dataframe_abril = dataframes["Abril"].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_abril = dataframe_abril.reset_index(inplace=False)
dataframe_abril = dataframe_abril.sort_values(by='Cantidad Pedida', ascending=False)
dataframe_abril

,Dia,Cantidad Pedida
20,21,761
17,18,737
16,17,733
12,13,729
3,4,710
0,1,702
23,24,702
13,14,702
10,11,702
9,10,699


In [41]:
fig = px.bar(dataframe_abril, x='Dia', y='Cantidad Pedida',
             title='Ventas por Día en Abril',
             labels={'Cantidad Pedida': 'Cantidad Vendida', 'Dia': 'Día'},
             color='Cantidad Pedida',  # Usar el valor de ventas para la paleta de colores
             color_continuous_scale='Viridis')  # Elegir una paleta de colores

# Ajustar el rango del eje y para resaltar mejor las diferencias
fig.update_yaxes(range=[0, dataframe_abril['Cantidad Pedida'].max() * 1.2])
fig.update_layout( xaxis=dict( tickvals= dataframe_abril['Dia']))
fig.show()

El dia de mayores ventas de Abril fue el 21.

In [42]:
dataframe_mayo = dataframes["Mayo"].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_mayo = dataframe_mayo.reset_index(inplace=False)
dataframe_mayo = dataframe_mayo.sort_values(by='Cantidad Pedida', ascending=False)
dataframe_mayo

,Dia,Cantidad Pedida
13,14,675
21,22,652
19,20,644
8,9,641
18,19,637
6,7,636
16,17,635
11,12,634
27,28,632
29,30,625


In [43]:
fig = px.bar(dataframe_mayo, x='Dia', y='Cantidad Pedida',
             title='Ventas por Día en Mayo',
             labels={'Cantidad Pedida': 'Cantidad Vendida', 'Dia': 'Día'},
             color='Cantidad Pedida',  # Usar el valor de ventas para la paleta de colores
             color_continuous_scale='Viridis')  # Elegir una paleta de colores

# Ajustar el rango del eje y para resaltar mejor las diferencias
fig.update_yaxes(range=[0, dataframe_mayo['Cantidad Pedida'].max() * 1.2])
fig.update_layout( xaxis=dict( tickvals= dataframe_mayo['Dia']))
fig.show()

El dia de mayores ventas de Mayo fue el 14.

In [44]:
dataframe_junio = dataframes["Junio"].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_junio = dataframe_junio.reset_index(inplace=False)
dataframe_junio = dataframe_junio.sort_values(by='Cantidad Pedida', ascending=False)
dataframe_junio

,Dia,Cantidad Pedida
28,29,548
5,6,542
18,19,538
14,15,537
13,14,536
25,26,531
4,5,530
15,16,529
20,21,529
19,20,528


In [45]:
fig = px.bar(dataframe_junio, x='Dia', y='Cantidad Pedida',
             title='Ventas por Día en Junio',
             labels={'Cantidad Pedida': 'Cantidad Vendida', 'Dia': 'Día'},
             color='Cantidad Pedida',  # Usar el valor de ventas para la paleta de colores
             color_continuous_scale='Viridis')  # Elegir una paleta de colores

# Ajustar el rango del eje y para resaltar mejor las diferencias
fig.update_yaxes(range=[0, dataframe_junio['Cantidad Pedida'].max() * 1.2])
fig.update_layout( xaxis=dict( tickvals= dataframe_junio['Dia']))
fig.show()

El dia de mayores ventas de Junio fue el 29.

In [46]:
dataframe_julio = dataframes["Julio"].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_julio = dataframe_julio.reset_index(inplace=False)
dataframe_julio = dataframe_julio.sort_values(by='Cantidad Pedida', ascending=False)
dataframe_julio

,Dia,Cantidad Pedida
14,15,573
25,26,557
17,18,554
21,22,546
8,9,541
30,31,540
0,1,539
7,8,538
24,25,536
19,20,536


In [47]:
fig = px.bar(dataframe_julio, x='Dia', y='Cantidad Pedida',
             title='Ventas por Día en Julio',
             labels={'Cantidad Pedida': 'Cantidad Vendida', 'Dia': 'Día'},
             color='Cantidad Pedida',  # Usar el valor de ventas para la paleta de colores
             color_continuous_scale='Viridis')  # Elegir una paleta de colores

# Ajustar el rango del eje y para resaltar mejor las diferencias
fig.update_yaxes(range=[0, dataframe_julio['Cantidad Pedida'].max() * 1.2])
fig.update_layout( xaxis=dict( tickvals= dataframe_julio['Dia']))
fig.show()

El dia de mayores ventas de Julio fue el 15.

In [48]:
dataframe_agosto = dataframes["Agosto"].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_agosto = dataframe_agosto.reset_index(inplace=False)
dataframe_agosto = dataframe_agosto.sort_values(by='Cantidad Pedida', ascending=False)
dataframe_agosto

,Dia,Cantidad Pedida
18,19,473
13,14,468
30,31,467
9,10,467
27,28,463
12,13,461
21,22,452
28,29,451
5,6,449
15,16,446


In [49]:
fig = px.bar(dataframe_agosto, x='Dia', y='Cantidad Pedida',
             title='Ventas por Día en Agosto',
             labels={'Cantidad Pedida': 'Cantidad Vendida', 'Dia': 'Día'},
             color='Cantidad Pedida',  # Usar el valor de ventas para la paleta de colores
             color_continuous_scale='Viridis')  # Elegir una paleta de colores

# Ajustar el rango del eje y para resaltar mejor las diferencias
fig.update_yaxes(range=[0, dataframe_agosto['Cantidad Pedida'].max() * 1.2])
fig.update_layout( xaxis=dict( tickvals= dataframe_agosto['Dia']))
fig.show()

El dia de mayores ventas de Agosto fue el 19.

In [50]:
dataframe_septiembre = dataframes["Septiembre"].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_septiembre = dataframe_septiembre.reset_index(inplace=False)
dataframe_septiembre = dataframe_septiembre.sort_values(by='Cantidad Pedida', ascending=False)
dataframe_septiembre

,Dia,Cantidad Pedida
26,27,472
18,19,468
14,15,462
28,29,457
10,11,456
16,17,454
1,2,452
13,14,452
11,12,450
0,1,447


In [51]:
fig = px.bar(dataframe_septiembre, x='Dia', y='Cantidad Pedida',
             title='Ventas por Día en Septiembre',
             labels={'Cantidad Pedida': 'Cantidad Vendida', 'Dia': 'Día'},
             color='Cantidad Pedida',  # Usar el valor de ventas para la paleta de colores
             color_continuous_scale='Viridis')  # Elegir una paleta de colores

# Ajustar el rango del eje y para resaltar mejor las diferencias
fig.update_yaxes(range=[0, dataframe_septiembre['Cantidad Pedida'].max() * 1.2])
fig.update_layout( xaxis=dict( tickvals= dataframe_septiembre['Dia']))
fig.show()



El dia de mayores ventas de Septiembre fue el 27.

In [52]:
dataframe_octubre = dataframes["Octubre"].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_octubre = dataframe_octubre.reset_index(inplace=False)
dataframe_octubre = dataframe_octubre.sort_values(by='Cantidad Pedida', ascending=False)
dataframe_octubre

,Dia,Cantidad Pedida
25,26,808
21,22,770
18,19,770
1,2,767
4,5,766
30,31,760
22,23,759
9,10,756
6,7,751
27,28,745


In [53]:
fig = px.bar(dataframe_octubre, x='Dia', y='Cantidad Pedida',
             title='Ventas por Día en Octubre',
             labels={'Cantidad Pedida': 'Cantidad Vendida', 'Dia': 'Día'},
             color='Cantidad Pedida',  # Usar el valor de ventas para la paleta de colores
             color_continuous_scale='Viridis')  # Elegir una paleta de colores

# Ajustar el rango del eje y para resaltar mejor las diferencias
fig.update_yaxes(range=[0, dataframe_octubre['Cantidad Pedida'].max() * 1.2])
fig.update_layout( xaxis=dict( tickvals= dataframe_octubre['Dia']))
fig.show()

El dia de mayores ventas de Octubre fue el 26.

In [54]:
dataframe_noviembre = dataframes["Noviembre"].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_noviembre = dataframe_noviembre.reset_index(inplace=False)
dataframe_noviembre = dataframe_noviembre.sort_values(by='Cantidad Pedida', ascending=False)
dataframe_noviembre

,Dia,Cantidad Pedida
20,21,718
0,1,714
7,8,710
3,4,700
23,24,700
6,7,699
26,27,688
18,19,688
12,13,684
17,18,673


In [55]:
fig = px.bar(dataframe_noviembre, x='Dia', y='Cantidad Pedida',
             title='Ventas por Día en Noviembre',
             labels={'Cantidad Pedida': 'Cantidad Vendida', 'Dia': 'Día'},
             color='Cantidad Pedida',  # Usar el valor de ventas para la paleta de colores
             color_continuous_scale='Viridis')  # Elegir una paleta de colores

# Ajustar el rango del eje y para resaltar mejor las diferencias
fig.update_yaxes(range=[0, dataframe_noviembre['Cantidad Pedida'].max() * 1.2])
fig.update_layout( xaxis=dict( tickvals= dataframe_noviembre['Dia']))
fig.show()

El dia de mayores ventas de Noviembre fue el 21.

In [56]:
dataframe_diciembre = dataframes["Diciembre"].groupby('Dia')['Cantidad Pedida'].sum()
dataframe_diciembre = dataframe_diciembre.reset_index(inplace=False)
dataframe_diciembre = dataframe_diciembre.sort_values(by='Cantidad Pedida', ascending=False)
dataframe_diciembre

,Dia,Cantidad Pedida
16,17,1044
22,23,988
12,13,957
28,29,954
18,19,942
27,28,931
29,30,928
25,26,925
13,14,924
11,12,919


In [57]:
fig = px.bar(dataframe_diciembre, x='Dia', y='Cantidad Pedida',
             title='Ventas por Día en Diciembre',
             labels={'Cantidad Pedida': 'Cantidad Vendida', 'Dia': 'Día'},
             color='Cantidad Pedida',  # Usar el valor de ventas para la paleta de colores
             color_continuous_scale='Viridis')  # Elegir una paleta de colores

# Ajustar el rango del eje y para resaltar mejor las diferencias
fig.update_yaxes(range=[0, dataframe_diciembre['Cantidad Pedida'].max() * 1.2])
fig.update_layout( xaxis=dict( tickvals= dataframe_diciembre['Dia']))
fig.show()

El dia de mayores ventas de Diciembre fue el 17.



---



Ventas segun el dia de la semana

In [58]:
#Hacemos una funcion para traducir los dias de la semana de Ingles a Español.

import pandas as pd
import plotly.express as px
def traducir_dia(dia):
    dias_en_ingles = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    dias_en_espanol = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
    return dias_en_espanol[dias_en_ingles.index(dia)]






In [59]:
#Generamos una columna nueva que nos muestre el dia de la semana
df_ventas_anual_desglosado['DiaSemana'] = pd.to_datetime(df_ventas_anual_desglosado['Dia'], format='%d').dt.day_name()


In [60]:
df_ventas_anual_desglosado

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Fecha de Compra,Hora,Dia,Mes,Calle,Ciudad,PO BOX,Estado,Cod Postal,DiaSemana
0,141234,iPhone,1,700,2019-01-22 21:25:00,"944 Walnut St, Boston, MA 02215",2019-01-22 21:25:00,21,22,1,944 Walnut St,Boston,MA 02215,MA,02215,Monday
1,141235,Lightning Charging Cable,1,14,2019-01-28 14:15:00,"185 Maple St, Portland, OR 97035",2019-01-28 14:15:00,14,28,1,185 Maple St,Portland,OR 97035,OR,97035,Sunday
2,141236,Wired Headphones,2,11,2019-01-17 13:33:00,"538 Adams St, San Francisco, CA 94016",2019-01-17 13:33:00,13,17,1,538 Adams St,San Francisco,CA 94016,CA,94016,Wednesday
3,141237,27in FHD Monitor,1,149,2019-01-05 20:33:00,"738 10th St, Los Angeles, CA 90001",2019-01-05 20:33:00,20,5,1,738 10th St,Los Angeles,CA 90001,CA,90001,Friday
4,141238,Wired Headphones,1,11,2019-01-25 11:59:00,"387 10th St, Austin, TX 73301",2019-01-25 11:59:00,11,25,1,387 10th St,Austin,TX 73301,TX,73301,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25112,319666,Lightning Charging Cable,1,14,2019-12-11 20:58:00,"14 Madison St, San Francisco, CA 94016",2019-12-11 20:58:00,20,11,12,14 Madison St,San Francisco,CA 94016,CA,94016,Thursday
25113,319667,AA Batteries (4-pack),2,3,2019-12-01 12:01:00,"549 Willow St, Los Angeles, CA 90001",2019-12-01 12:01:00,12,1,12,549 Willow St,Los Angeles,CA 90001,CA,90001,Monday
25114,319668,Vareebadd Phone,1,400,2019-12-09 06:43:00,"273 Wilson St, Seattle, WA 98101",2019-12-09 06:43:00,6,9,12,273 Wilson St,Seattle,WA 98101,WA,98101,Tuesday
25115,319669,Wired Headphones,1,11,2019-12-03 10:39:00,"778 River St, Dallas, TX 75001",2019-12-03 10:39:00,10,3,12,778 River St,Dallas,TX 75001,TX,75001,Wednesday


In [61]:
# Ahora generamos la misma columna pero con la funcion hecha para traducir el dia.
df_ventas_anual_desglosado['DiaSemana'] = pd.to_datetime(df_ventas_anual_desglosado['Fecha de Pedido']).dt.day_name().apply(traducir_dia)



In [62]:
df_ventas_anual_desglosado

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Fecha de Compra,Hora,Dia,Mes,Calle,Ciudad,PO BOX,Estado,Cod Postal,DiaSemana
0,141234,iPhone,1,700,2019-01-22 21:25:00,"944 Walnut St, Boston, MA 02215",2019-01-22 21:25:00,21,22,1,944 Walnut St,Boston,MA 02215,MA,02215,Martes
1,141235,Lightning Charging Cable,1,14,2019-01-28 14:15:00,"185 Maple St, Portland, OR 97035",2019-01-28 14:15:00,14,28,1,185 Maple St,Portland,OR 97035,OR,97035,Lunes
2,141236,Wired Headphones,2,11,2019-01-17 13:33:00,"538 Adams St, San Francisco, CA 94016",2019-01-17 13:33:00,13,17,1,538 Adams St,San Francisco,CA 94016,CA,94016,Jueves
3,141237,27in FHD Monitor,1,149,2019-01-05 20:33:00,"738 10th St, Los Angeles, CA 90001",2019-01-05 20:33:00,20,5,1,738 10th St,Los Angeles,CA 90001,CA,90001,Sábado
4,141238,Wired Headphones,1,11,2019-01-25 11:59:00,"387 10th St, Austin, TX 73301",2019-01-25 11:59:00,11,25,1,387 10th St,Austin,TX 73301,TX,73301,Viernes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25112,319666,Lightning Charging Cable,1,14,2019-12-11 20:58:00,"14 Madison St, San Francisco, CA 94016",2019-12-11 20:58:00,20,11,12,14 Madison St,San Francisco,CA 94016,CA,94016,Miércoles
25113,319667,AA Batteries (4-pack),2,3,2019-12-01 12:01:00,"549 Willow St, Los Angeles, CA 90001",2019-12-01 12:01:00,12,1,12,549 Willow St,Los Angeles,CA 90001,CA,90001,Domingo
25114,319668,Vareebadd Phone,1,400,2019-12-09 06:43:00,"273 Wilson St, Seattle, WA 98101",2019-12-09 06:43:00,6,9,12,273 Wilson St,Seattle,WA 98101,WA,98101,Lunes
25115,319669,Wired Headphones,1,11,2019-12-03 10:39:00,"778 River St, Dallas, TX 75001",2019-12-03 10:39:00,10,3,12,778 River St,Dallas,TX 75001,TX,75001,Martes


In [63]:
# Calculamos la suma de ventas por día de la semana
ventas_por_dia_semana = df_ventas_anual_desglosado.groupby('DiaSemana')['Cantidad Pedida'].sum().reset_index()


In [64]:
ventas_por_dia_semana

,DiaSemana,Cantidad Pedida
0,Domingo,29778
1,Jueves,29662
2,Lunes,29899
3,Martes,30724
4,Miércoles,29799
5,Sábado,29769
6,Viernes,29448


In [65]:
#Generamos el grafico de ventas segun el dia de la semana.
fig = px.bar(
    ventas_por_dia_semana,
    x='DiaSemana',
    y='Cantidad Pedida',
    text='Cantidad Pedida',
    title='Ventas anuales según el día de la semana',
    labels={'Cantidad Pedida': 'Cantidad Vendida', 'DiaSemana': 'Día de la Semana'},
    category_orders={"DiaSemana": ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]}
)

fig.update_traces(
    marker_color=['blue', 'green', 'red', 'purple', 'orange', 'pink', 'cyan'],
    texttemplate='%{text}',
    textposition='outside',
    marker_line_width=2,
    marker_line_color='black',
    opacity=0.8
)

fig.update_xaxes(title='Día de la semana')
fig.update_xaxes(categoryorder='total descending')
fig.update_yaxes(title='Ventas')

fig.show()

In [66]:
fig = px.pie(ventas_por_dia_semana, values='Cantidad Pedida', names='DiaSemana',
             title='Porcentaje de Ventas por Día de la Semana')

fig.show()